## Create the Web Application Skeleton

In [ ]:
%%writefile app.py
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config['SECRET_KEY'] = "secret-key-123"
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///data.db'

db = SQLAlchemy(app)

from routes import *

if __name__ == "__main__":
    app.run(debug=True)

## Create route modules in Web Application

In [ ]:
%%writefile routes.py
from app import app, db
from flask import render_template, redirect, url_for, flash, get_flashed_messages

from datetime import datetime

from models import Task
import forms

@app.route('/')
@app.route('/index')
def index():
    tasks = Task.query.all()
    return render_template("index.html",tasks=tasks)

@app.route("/add", methods=['GET', 'POST'])
def add():
    form = forms.AddTaskForm()
    if form.validate_on_submit():
        task = Task(title=form.title.data, date=datetime.utcnow())
        db.session.add(task)
        db.session.commit()
        flash("Task added")
        return redirect(url_for('index'))
    return render_template("add.html", form=form)

@app.route("/edit/<int:task_id>", methods=['GET', 'POST'])
def edit(task_id):
    form = forms.AddTaskForm()
    task = Task.query.get(task_id)
    print(task)
    
    if task:
        if form.validate_on_submit():
            task.title = form.title.data
            task.datetime = datetime.utcnow()
            db.session.commit()
            flash("Task updated")
            return redirect(url_for('index'))
        form.title.data = task.title
        return render_template("edit.html", form=form, task_id=task_id)
    else:
        flash(f'Task with id {task_id} does not exit!')
    return redirect(url_for('index'))

@app.route("/delete/<int:task_id>", methods=['GET', 'POST'])
def delete(task_id):
    form = forms.DeleteTaskForm()
    task = Task.query.get(task_id)
        
    if task:
        if form.validate_on_submit():
            if form.submit.data:
                db.session.delete(task)
                db.session.commit()
                flash(f'Task with id {task_id} deleted')
            return redirect(url_for('index'))
        return render_template("delete.html", form=form, task_id=task_id, title=task.title)
    else:
        flash(f'Task with id {task_id} does not exit!')

    return redirect(url_for('index'))

## Create the form for Web App

In [ ]:
%%writefile forms.py
from flask_wtf import FlaskForm
from wtforms import StringField, SubmitField
from wtforms.validators import DataRequired

class AddTaskForm(FlaskForm):
    title = StringField('Title', validators=[DataRequired()])
    submit = SubmitField('Submit')
    
class DeleteTaskForm(FlaskForm):
    submit = SubmitField('Delete')

## Create Data Model

In [ ]:
%%writefile models.py
from app import db

class Task(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(100), nullable=False)
    date = db.Column(db.Date, nullable=False )
    
    def __repr__(self):
        return f'{self.title} created on {self.date}'

## Script to intialize to create the db

In [ ]:
%%writefile script.py

from models import db

db.create_all()

from models import Task
from datetime import datetime

t = Task(title='xyz', date=datetime.utcnow())
print(t)

db.session.add(t)
db.session.commit()

tasks = Task.query.all()
tasks